<hr style="border:2px solid #808080"> </hr>
<center><h1 style="color:#03122E;"> Álgebra Lineal Numérica IMT2111</h1></center>
<center><h1 style="color:#173F8A;"> Capítulo 1: Eliminacion Gaussiana</h3></center>
<center><h1 style="color:#0176DE;"> Prof. Manuel A. Sánchez</h3></center>
<hr style="border:2px solid #808080"> </hr>

## Tabla de contenidos
1. Eliminacion Gaussiana
2. Ejemplo Worst-case instability

In [1]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
display(HTML("""<style>.output {display: flex;align-items: center;text-align: center;}</style>"""))

In [2]:
# Forward and Backward substitution
def forward_substitution(L,b):
    '''
    Forward substitution algorithm for system L x = b
    input : L lower triangular matrix n x n
            b vector n x 1
    output: x solution of L x = b
    '''
    n = L.shape[0]; x = np.zeros(n)
    x[0] = b[0]/L[0,0]
    for i in range(1,n):
        x[i] = (b[i] - L[i,0:i]@x[0:i])/L[i,i]
    return x
def backward_substitution(U,b):
    '''
    Backward substitution algorithm for system U x = b
    input : U upper tringular matrix n x n
            b vector n x 1
    output : x solution of U x = b
    '''
    n = U.shape[0]; x = np.zeros(n)
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - U[i,(i+1):n]@(x[(i+1):n]))/U[i,i]
    return x
# generate permutation matrix
def perm(rowpiv):
    n = rowpiv.size+1
    P = np.eye(n)
    for k in range(n-1):
        P[[k,rowpiv[k]],:] = P[[rowpiv[k], k],:]
    return P
def permb(b, rowpiv):
    n = b.size
    for k in range(n-1):
        b[[k, rowpiv[k]]] = b[[rowpiv[k],k]] 
    return b

In [6]:
# 1.
def GE(Ainput, binput):
    '''
    Eliminacion Gaussiana sin pivoteo
    Input : A nonsingular and square matrix n x n 
            b vector n x 1
    Output: x solution of the system A x = b
    '''
    A = Ainput.copy() # se va a modificar
    b = binput.copy() # se va a modificar
    # 1. Factorize A = LU
    L, U = LU(A)
    # 2. Solve LUx = b forward substitution
    y = forward_substitution(L, b)
    # 3. Solve Ux = L^{-1} b backward substitution
    x = backward_substitution(U,y)
    return x
def LU(Ainput):
    '''
    Factorizacion A = LU, sin pivoteo
    Input : A nonsingular and square matrix n x n 
    Output: L triangular inferior, U triangular superior, square matrix n x n
    '''
    A = Ainput.copy() # se va a modificar
    n = A.shape[0]
    for i in range(n-1):
        if A[i, i] == 0:
            raise ValueError("coeficient is zero.")
        A[(i+1):n,i] = (A[(i+1):n,i]/A[i,i])
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U

# 2.
def GEPP(Ainput, binput):
    A = Ainput.copy() # se va a modificar
    b = binput.copy() # se va a modificar
    '''
    Eliminacion Gaussiana con pivoteo de filas o pivoteo parcial
    Input : A matriz cuadrada no singular de n x n 
            b vector de n x 1
    Output: x solucion del sistema lineal  A x = b
    '''
    # 1. Factorize A = PLU
    L, U, pT = LUPP(A)
    # 2. Solve P L U x = b
    P = perm(pT)
    rhs  = perm(pT).dot(b)
    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, rhs)
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    return x

def LUPP(Ainput):
    '''
    Factorizacion PA = LU, con pivoteo parcial
    Input : A matriz cuadrada no singular de n x n
    Output: L triangular inferior, matriz cuadrada de n x n
            U trangular superior, matriz cuadrada de n x n
            p, vector asociado a la matriz de permutacion
    '''
    A = Ainput.copy() # se va a modificar
    n = A.shape[0]
    piv = np.arange(0,n-1)
    for i in range(n-1):
        imax = abs(A[i:,i]).argmax() + i
        piv[i] = imax
        if A[imax, i] == 0:
            raise ValueError("Matrix is singular.")
        elif imax != i:
            A[[i,imax],:] = A[[imax, i],:][:]
        A[(i+1):n,i][:] = (A[(i+1):n,i]/A[i,i])[:]
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, piv

# 
def GECP(Ainput, binput):
    '''
    Eliminacion Gaussiana con pivoteo de filas y columnas o pivoteo completo
    Input : A matriz cuadrada no singular de n x n 
            b vector de n x 1
    Output: x solucion del sistema lineal  A x = b
    '''
    A = Ainput.copy() # se va a modificar
    b = binput.copy() # se va a modificar
    # 1. Factorize PAQ^T = LU
    L, U, rowpiv, colpiv = LUCP(A)
    # 2. Solve  L U x = P b Q^T
    rhs  = perm(rowpiv).dot(b)
    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, rhs)
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    return (perm(colpiv).T).dot(x)

def LUCP(Ainput):
    '''
    Factorizacion P A Q = L U, con pivoteo completo
    Input : A matriz cuadrada no singular de n x n
    Output: L triangular inferior, matriz cuadrada de n x n
            U trangular superior, matriz cuadrada de n x n
            p, vector asociado a la matriz de permutacion por filas
            q, vector asociado a la matriz de permutacion por columnas
    '''
    A = Ainput.copy() # se va a modificar
    n = A.shape[0]
    rowpiv = np.arange(0,n-1)
    colpiv = np.arange(0,n-1)
    for i in range(n-1):
        mu, lam = np.unravel_index(np.argmax(np.abs(A[i:,i:]), axis=None), A[i:,i:].shape)
        mu +=i; lam+=i
        if A[mu, lam] == 0:
            raise ValueError("Matrix is singular.")
        else:
            rowpiv[i] = mu
            A[[i, mu],:] = A[[mu, i],:][:]
            colpiv[i] = lam
            A[:,[i, lam]] = A[:,[lam, i]][:]
        A[(i+1):n,i] *= 1.0/A[i,i]
        A[(i+1):n, (i+1):n] -=np.outer(A[(i+1):n,i],A[i, (i+1):n])
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, rowpiv, colpiv

In [3]:
def Eliminacion_Gaussiana(A,b, pivoteo=None):
    if pivoteo is None:
        x = GE(A,b)
    elif pivoteo == 'parcial':
        x = GEPP(A,b)
    elif pivoteo == 'completo':
        x = GECP(A,b)
    return x

def Factorizacion_LU(A, pivoteo=None):
    if pivoteo is None:
        L,U = LU(A)
        return L,U
    elif pivoteo == 'parcial':
        L,U,piv = LUPP(A)
        return L, U, piv
    elif pivoteo == 'completo':
        L, U, rowpiv, colpiv = LUCP(A)
        return  L, U, rowpiv, colpiv

## Ejemplo: eliminacion Gaussiana

Considere el sistema lineal
\begin{equation}
Ax = b,\qquad 
A = \begin{bmatrix}
1 & 3 & 4 & 1\\
2 & 1 & 5 & 1 \\
3 & 1 & 6 & 1 \\
6 & 2 & 3 & 2
\end{bmatrix},\quad
b = \begin{bmatrix}
-2 \\ -2\\ -2 \\ 5
\end{bmatrix}
\end{equation}
Resuelva usando Eliminacion Gaussiana, i) sin pivoteo, ii) con pivoteo parcial, iii) con pivoteo completo.

In [7]:
A = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)
x = np.array([1, 0, -1,1])
b = np.array([-2,-2,-2,5],dtype=np.float64)

In [11]:
x_GE   = Eliminacion_Gaussiana(A,b)
x_GEPP = Eliminacion_Gaussiana(A,b, pivoteo='parcial')
x_GECP = Eliminacion_Gaussiana(A,b, pivoteo='completo')
print("x_GE   : ", x_GE)
print("x_GEPP : ", x_GEPP)
print("x_GECP : ", x_GECP)
print("x      : ", np.linalg.solve(A,b))


x_GE   :  [ 1.00000000e+00  2.66453526e-16 -1.00000000e+00  1.00000000e+00]
x_GEPP :  [ 1.  0. -1.  1.]
x_GECP :  [ 1.00000000e+00  3.74700271e-16 -1.00000000e+00  1.00000000e+00]
x      :  [ 1.  0. -1.  1.]


## Ejemplo: Factorizacion LU

Encuentre la factorizacion LU de la siguiente matriz usando los algoritmos de Eliminacion Gaussiana, i) sin pivoteo, ii) con pivoteo parcial, iii) con pivoteo completo. Compare las matrices:
\begin{equation}
LU = A,\qquad 
A = \begin{bmatrix}
1 & 3 & 4 & 1\\
2 & 1 & 5 & 1 \\
3 & 1 & 6 & 1 \\
6 & 2 & 3 & 2
\end{bmatrix},
\end{equation}

In [9]:
A = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)

In [10]:
L, U = Factorizacion_LU(A, pivoteo=None)
print("L :\n", L)
print("U :\n", U)

L :
 [[1.  0.  0.  0. ]
 [2.  1.  0.  0. ]
 [3.  1.6 1.  0. ]
 [6.  3.2 9.5 1. ]]
U :
 [[ 1.   3.   4.   1. ]
 [ 0.  -5.  -3.  -1. ]
 [ 0.   0.  -1.2 -0.4]
 [ 0.   0.   0.   3. ]]


In [118]:
L, U, p = Factorizacion_LU(A, pivoteo='parcial')
print("L :\n", L)
print("U :\n", U)

L :
 [[1.         0.         0.         0.        ]
 [0.16666667 1.         0.         0.        ]
 [0.5        0.         1.         0.        ]
 [0.33333333 0.125      0.79166667 1.        ]]
U :
 [[6.         2.         3.         2.        ]
 [0.         2.66666667 3.5        0.66666667]
 [0.         0.         4.5        0.        ]
 [0.         0.         0.         0.25      ]]


In [119]:
L, U,p,q = Factorizacion_LU(A, pivoteo='completo')
print("L :\n", L)
print("U :\n", U)

L :
 [[ 1.          0.          0.          0.        ]
 [ 0.5         1.          0.          0.        ]
 [ 0.66666667 -0.22222222  1.          0.        ]
 [ 0.83333333 -0.11111111  0.125       1.        ]]
U :
 [[6.         3.         1.         1.        ]
 [0.         4.5        1.5        1.5       ]
 [0.         0.         2.66666667 0.66666667]
 [0.         0.         0.         0.25      ]]


## Ejemplo: inestabilidad sin pivoteo
\begin{equation}
A = \begin{bmatrix}
0 & 1 \\
1 & 1
\end{bmatrix}, \quad \text{Eliminacion Gaussiana falla en el primer paso con } \kappa(A) = (3+\sqrt{5})/2
\end{equation}

\begin{equation}
\tilde{A} = \begin{bmatrix}
10^{-20} & 1 \\
1 & 1
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 \\
10^{20} & 1
\end{bmatrix}
\begin{bmatrix}
10^{-20} & 1 \\
0 & 1-10^{20}
\end{bmatrix}, \quad \text{Eliminacion Gaussiana ahora no falla}
\end{equation}

In [195]:
Afalla = np.array([[0,1],[1,1]], dtype=np.float64)
Atilde = np.array([[10**(-20), 1],[1,1]], dtype=np.float64)

In [196]:
L,U = Factorizacion_LU(Afalla)


ValueError: coeficient is zero.

In [197]:
L,U = Factorizacion_LU(Atilde)
print(L)
print(U)
print(L@U- Atilde)

[[1.e+00 0.e+00]
 [1.e+20 1.e+00]]
[[ 1.e-20  1.e+00]
 [ 0.e+00 -1.e+20]]
[[ 0.  0.]
 [ 0. -1.]]


In [198]:
b = np.array([1,0])
xtilde = Eliminacion_Gaussiana(Atilde,b)
print("solucion calculada:", xtilde)
print("la solucion exacta es x = [-1,1]!")

solucion calculada: [0. 1.]
la solucion exacta es x = [-1,1]!


**Nota** La factorizacion LU es estable, no backward stable. Pero al usar eliminacion Gaussiana sin pivoteo para resolver $Ax=b$ no lo hace estable.

En general, si un paso del algoritmo es estable pero backward stable para resolver un subproblema, entonces la estabilidad del algortimo puede estar en peligro.

**Nota** Complejidad:  eliminacion Gaussiana sin pivoteo es de $\approx \frac{2}{3} m^{3}$ flops.

## Ejemplo: worst case

## Inestabilidad

Para ciertas matrices $A$ a pesar de os efectos beneficiosos de pivotear, el factor de crecimiento $\rho$ se vuelve gigante. Por ejemplo, suponga que la matriz $A$ tiene la siguiente forma:

$$
A = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 \\
-1 &1 &0 & 0& 1 \\
-1& -1 &1 & 0 &1\\
-1&-1 &-1 & 1 & 1 \\
-1& -1 &-1 &-1 &1
\end{bmatrix}
$$

En este caso, la factorization da 

$$
U = \begin{bmatrix}
 1. & 0. & 0. & 0.&  1.\\
 0. & 1. & 0. & 0.&  2.\\
 0. & 0. & 1. & 0.&  4.\\
 0. & 0. & 0. & 1.&  8.\\
 0. & 0. & 0. & 0.& 16.
 \end{bmatrix}
$$

para esta matriz de $n\times n$, con $n=5$, el factor de crecimiento es $\rho = 2^{n-1} = 16$.

Un factor de crecimiento de orden $2^{n}$ corresponde a una perdida del orden de $n$ bits de precision, lo cual es catastrofico para computaciones practicas. Como un computador standard representa numeros de punto flotante con solo 64 bits, con matries de dimensiones en cientos y miles en dimension perder m bits de precision es intolerable.

In [102]:
n = 4
A_wc = -np.tril(np.ones((n,n)),-1)+np.eye(n)
A_wc[:,-1] = np.ones(n)
A_wc

array([[ 1.,  0.,  0.,  1.],
       [-1.,  1.,  0.,  1.],
       [-1., -1.,  1.,  1.],
       [-1., -1., -1.,  1.]])

In [103]:
L, U, pT = Factorizacion_LU(A_wc, pivoteo='parcial')
U

array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [0., 0., 1., 4.],
       [0., 0., 0., 8.]])

In [104]:
L, U, pT, q = Factorizacion_LU(A_wc, pivoteo='completo')
U

array([[ 1.,  1.,  0.,  0.],
       [ 0.,  2.,  1.,  0.],
       [ 0.,  0., -2.,  1.],
       [ 0.,  0.,  0., -2.]])

# Example. 
Consider the system $Ax = b$ with 
\begin{equation}
A = 
\begin{pmatrix}
6 & 3 & 4 & 1 \\ 2 & 1 & 5 & 1 \\ 1 & 1 & 6 & 1 \\ 0 & 2 & 3 & 2
\end{pmatrix}
\qquad
b = 
\begin{pmatrix}
3\\2\\1\\3
\end{pmatrix}
\end{equation}

The solution of the system is
\begin{equation}
x= 
\begin{pmatrix}
-8/9\\0\\-1/9\\13/3
\end{pmatrix}
\end{equation}

In [105]:
A0 = np.array([[3,2,6],
               [2,0,5],
               [6,2,3]
               ],dtype=np.float64)
b0 = np.array([1,2,3], dtype=np.float64)
L,U,rowpiv = LUPP(A0)

# print("LU", L@U)
# print("PA",perm(rowpiv)@A0)
x_np = np.linalg.solve(A0,b0)

x_GEPP = GEPP(A0,b0)
x_GE = GE(A0,b0)

dferror = {'Errors:':['||x-xaprox||','||b-Axaprox||'], 'Gaussian Ellimination':[np.linalg.norm(x_np-x_GE), np.linalg.norm(b0-A0.dot(x_GE))], 'Gaussian Ellimination with PP':[np.linalg.norm(x_np-x_GEPP),  np.linalg.norm(b0-A0.dot(x_GEPP))]}
df = pd.DataFrame(data=dferror)
df
# print("Soluciones:\n")
# print(x_np )
# print(x_GE )
# print(x_GEPP )


[2 2]
[2 2]


,Errors:,Gaussian Ellimination,Gaussian Ellimination with PP
0,||x-xaprox||,1.387779e-17,0.000000e+00
1,||b-Axaprox||,4.965068e-16,4.577567e-16


# Example. 
Consider the system $Ax = b$ with 
\begin{equation}
A = 
\begin{pmatrix}
1 & 3 & 4 & 1 \\ 2 & 1 & 5 & 1 \\ 4 & 1 & 6 & 1 \\ 6 & 2 & 3 & 2
\end{pmatrix}
\qquad
b = 
\begin{pmatrix}
3\\2\\1\\3
\end{pmatrix}
\end{equation}

The solution of the system is
\begin{equation}
x= 
\begin{pmatrix}
-8/9\\0\\-1/9\\13/3
\end{pmatrix}
\end{equation}

In [202]:
A0 = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)
b0 = np.array([3,2,1,3], dtype=np.float64)
# L,U,rowpiv, colpiv = LUCP(A0)
# print(L@U)
# print(perm(rowpiv)@A0@perm(colpiv).T)

x_GECP = GECP(A0,b0)
x_GEPP = GEPP(A0,b0)
x_GE = GE(A0,b0)
x_np = np.linalg.solve(A0,b0)

print("Soluciones:\n")
print(x_np )
print(x_GE )
print(x_GEPP )
print(x_GECP )

U:
 [[6.         3.         1.         1.        ]
 [0.         4.5        1.5        1.5       ]
 [0.         0.         2.66666667 0.66666667]
 [0.         0.         0.         0.25      ]]
[2 3 2]
GECP [-1.11111111e-01 -8.88888889e-01 -3.33066907e-16  4.33333333e+00]
[2 2 2]
[3. 2. 1. 3.]
U:
 [[6.         2.         3.         2.        ]
 [0.         2.66666667 3.5        0.66666667]
 [0.         0.         4.5        0.        ]
 [0.         0.         0.         0.25      ]]
[3 3 2]
[3 3 2]
Soluciones:

[-0.88888889  0.         -0.11111111  4.33333333]
[-0.88888889 -0.         -0.11111111  4.33333333]
[-0.88888889  0.         -0.11111111  4.33333333]
[-8.88888889e-01 -3.33066907e-16 -1.11111111e-01  4.33333333e+00]


In [203]:

dferror = {'Errors:':['||x-xaprox||','||b-Axaprox||'], 
           'Gaussian Ellimination':[np.linalg.norm(x_np-x_GE), np.linalg.norm(b0-A0.dot(x_GE))], 
           'Gaussian Ellimination with PP':[np.linalg.norm(x_np-x_GEPP),  np.linalg.norm(b0-A0.dot(x_GEPP))],
           'Gaussian Ellimination with CP':[np.linalg.norm(x_np-x_GECP),  np.linalg.norm(b0-A0.dot(x_GECP))]}
df = pd.DataFrame(data=dferror)

df

,Errors:,Gaussian Ellimination,Gaussian Ellimination with PP,Gaussian Ellimination with CP
0,||x-xaprox||,4.652682e-16,1.110223e-16,9.501979e-16
1,||b-Axaprox||,2.843558e-15,9.930137e-16,1.935740e-15


In [108]:
e1 = []; e2 = []; e3 = []; e4 = []
N = [5*i for i in range(1,8)]
for nn in N:
    A_wc = -np.tril(np.ones((nn,nn)),-1)+np.eye(nn)
    A_wc[:,-1] = np.ones(nn)    
    x = np.random.rand(nn); 
    b = A_wc.dot(x)

    x_gepp = Eliminacion_Gaussiana(A_wc, b, pivoteo='parcial')
    x_gecp = Eliminacion_Gaussiana(A_wc, b, pivoteo='completo')

    e1.append(np.linalg.norm(x-x_gepp))
    e2.append(np.linalg.norm(x-x_gecp))
    e3.append(np.linalg.norm(b-A_wc.dot(x_gepp)))
    e4.append(np.linalg.norm(b-A_wc.dot(x_gecp)))

dferror = {'n':N, '||x-x_gepp||':e1, '||x-x_gecp||':e2, '||b-A x_gepp||':e3, '||b - A x_gecp||':e4}
df = pd.DataFrame(data=dferror)
df

[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 4 4 4]
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
[0 9 9 9 9 9 9 9 9]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0 14 14 14 14 14 14 14 14 14 14 14 14 14]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26

,n,||x-x_gepp||,||x-x_gecp||,||b-A x_gepp||,||b - A x_gecp||
0,5,3.682193e-16,1.922963e-16,4.440892e-16,0.000000e+00
1,10,3.158521e-14,4.440892e-16,4.998150e-14,1.110223e-16
2,15,2.136576e-13,6.106227e-16,3.662814e-13,2.220446e-16
3,20,7.711734e-12,7.152112e-16,1.359018e-11,2.195324e-15
4,25,3.695089e-10,1.420695e-15,8.300233e-10,3.463113e-15
5,30,4.840548e-10,1.633399e-15,8.045072e-10,2.189746e-15
6,35,2.466247e-07,3.202373e-15,5.058859e-07,7.794525e-15


In [64]:
e1 = []; e2 = []
N = [5*i for i in range(1,3)]
for nn in N:
    A_wc = -np.tril(np.ones((nn,nn)),-1)+np.eye(nn)
    A_wc[:,-1] = np.ones(nn)
    xsol = np.ones(nn) # np.random.rand(nn); 
    b = A_wc.dot(xsol)
    
    x_gecp = Eliminacion_Gaussiana(A_wc,b, pivoteo='completo')
    # x_gecp = gaussian_elimination(A_wc, b)
    print(A_wc.dot(x_gecp)-b)
    e1.append(np.linalg.norm(xsol-x_gecp))
    e2.append(np.linalg.norm(b-A_wc.dot(x_gecp)))

dferror = {'n':N, '||x-x_gecp||':e1, '||b-A*x_gecp||':e2}
df = pd.DataFrame(data=dferror)
df

[0 1 2 3]
[0 4 2 3]
[0. 0. 0. 0. 0.]
[0 1 2 3 4 5 6 7 8]
[0 9 2 3 4 5 6 7 8]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


,n,||x-x_gecp||,||b-A*x_gecp||
0,5,0.0,0.0
1,10,0.0,0.0


In [1]:
def gaussian_elimination_complete_pivoting(A, b):
    """
    Perform Gaussian elimination with complete pivoting on the augmented matrix [A | b].
    Returns the solution if it exists, or None if no solution exists.
    """
    n = len(A)
    
    # Augment the matrix A with the vector b
    M = [row + [bi] for row, bi in zip(A, b)]

    # Forward elimination
    for i in range(n):
        # Complete pivoting: Find the pivot element
        max_val = -1
        pivot_row, pivot_col = None, None
        for row in range(i, n):
            for col in range(i, n):
                if abs(M[row][col]) > max_val:
                    max_val = abs(M[row][col])
                    pivot_row, pivot_col = row, col
        
        if pivot_row is None or pivot_col is None:
            return None  # No unique solution exists
        
        M[i], M[pivot_row] = M[pivot_row], M[i]
        for row in range(n):
            M[row][i], M[row][pivot_col] = M[row][pivot_col], M[row][i]

        # Make the diagonal element 1
        divisor = M[i][i]
        if divisor == 0:
            return None  # No unique solution exists
        M[i] = [elem / divisor for elem in M[i]]

        # Eliminate the ith variable from the lower rows
        for j in range(i + 1, n):
            multiplier = M[j][i]
            M[j] = [elem_j - elem_i * multiplier for elem_i, elem_j in zip(M[i], M[j])]

    # Back substitution
    x = [0] * n
    for i in range(n - 1, -1, -1):
        x[i] = M[i][-1]
        for j in range(i + 1, n):
            x[i] -= M[i][j] * x[j]

    return x

# Example usage:
A = [[2, 1, -1], [-3, -1, 2], [-2, 1, 2]]
b = [8, -11, -3]
solution = gaussian_elimination_complete_pivoting(A, b)
if solution is None:
    print("No unique solution exists.")
else:
    print("Solution:", solution)


Solution: [2.0, 3.0, -0.9999999999999999]


In [57]:
def gaussian_elimination(A, b):
    """
    Perform Gaussian elimination on the augmented matrix [A | b].
    Returns the solution if it exists, or None if no solution exists.
    """
    n = len(A)
    
    # Augment the matrix A with the vector b
    M = [row + [bi] for row, bi in zip(A, b)]

    # Forward elimination
    for i in range(n):
        # Partial pivoting
        max_row = max(range(i, n), key=lambda k: abs(M[k][i]))
        M[i], M[max_row] = M[max_row], M[i]

        # Make the diagonal element 1
        divisor = M[i][i]
        if divisor == 0:
            return None  # No unique solution exists
        M[i] = [elem / divisor for elem in M[i]]

        # Eliminate the ith variable from the lower rows
        for j in range(i + 1, n):
            multiplier = M[j][i]
            M[j] = [elem_j - elem_i * multiplier for elem_i, elem_j in zip(M[i], M[j])]

    # Back substitution
    x = [0] * n
    for i in range(n - 1, -1, -1):
        x[i] = M[i][-1]
        for j in range(i + 1, n):
            x[i] -= M[i][j] * x[j]

    return x

# Example usage:
A = [[2, 1, -1], [-3, -1, 2], [-2, 1, 2]]
b = [8, -11, -3]
solution = gaussian_elimination(A, b)
if solution is None:
    print("No unique solution exists.")
else:
    print("Solution:", solution)


Solution: [2.0, 3.0, -0.9999999999999999]


In [3]:
A_partial = [[2, 1, -1], [-3, -1, 2], [-2, 1, 2]]
b_partial = [1, -1, 7]

solution_partial = gaussian_elimination(A_partial, b_partial)
print("Solution with partial pivoting:", solution_partial)


Solution with partial pivoting: [-6.0000000000000036, 7.000000000000003, -6.000000000000004]


In [57]:
import numpy as np

def cramer_rule(Ainput, binput):
    A = Ainput.copy()
    b = binput.copy()
    """
    Solve the system of linear equations Ax = b using Cramer's Rule.
    Returns the solution vector x.
    """
    n = len(A)
    det_A = np.linalg.det(A)
    if np.isclose(det_A, 0, atol = 10**(-20)):
        raise ValueError("Matrix A is singular, Cramer's Rule cannot be applied.")

    x = np.zeros(n)
    for i in range(n):
        Ai = A.copy()
        Ai[:, i] = b
        det_Ai = np.linalg.det(Ai)
        x[i] = det_Ai / det_A

    return x

# Example usage:
A = np.array([[2, 1, -1], [-3, -1, 2], [-2, 1, 2]], dtype=np.float64)
x = np.array([1,1,1], dtype=np.float64)
b = A.dot(x)
# b = np.array([8, -11, -3])

x_cr = cramer_rule(A, b)
x_gepp = Eliminacion_Gaussiana(A,b, pivoteo='parcial')
print("Solution using Cramer's Rule:", x_cr)
print("Solution using GE:", x_gepp, b - A.dot(x_gepp))
print("Solution using np:", np.linalg.solve(A,b))

Solution using Cramer's Rule: [1. 1. 1.]
Solution using GE: [1. 1. 1.] [0.0000000e+00 4.4408921e-16 4.4408921e-16]
Solution using np: [1. 1. 1.]


In [63]:
A = np.array([[0.9999999999999, 3],[0.5, 1.5]], dtype=np.float64)
print(np.linalg.det(A))
x = np.array([3,1.5], dtype=np.float64)
b = A.dot(x)
# b = np.array([10**4,-10**(-4)], dtype=np.float64)
print(b)
print(" cond(A) = ", np.linalg.cond(A))
x_gepp = Eliminacion_Gaussiana(A,b)
x_cr = cramer_rule(A, b)


e_cr   = np.linalg.norm(b-A.dot(x_cr  ))/(np.linalg.norm(x_cr  )*np.linalg.norm(A))
e_cepp = np.linalg.norm(b-A.dot(x_gepp))/(np.linalg.norm(x_gepp)*np.linalg.norm(A))
print(e_cr, e_gepp)

-1.5010215292930597e-13
[7.5  3.75]
 cond(A) =  83363865151024.78
0.00041873634397408236 8.241633834635454e-11
